##**[DACON] 축구 경기 결과 예측 AI 해커톤**

###**주제 및 데이터 설명**

**대회 설명**
- 주제: 축구경기와 관련된 데이터를 활용하여 축구 경기 결과 예측
- 설명: 축구 경기와 관련된 데이터를 활용하여 가능한 축구 경기 결과(원정팀 승리, 무승부, 홈팀 승리)의 확률을 각각 예측

**데이터**

train.csv
-  EPL(England Premier League) 경기에서 발생한 경기 관련 데이터
  - 1. matchID: 경기 고유 ID
  - 2. season: 경기가 진행된 시즌
  - 3. date: 경기가 진행된 일자 (일부 데이터의 경우 데이터 수집환경에 따라 실제 영국 시간과 방생한 경기 차이가 존재함)
  - 4. homeTeam: 홈팀
  - 5. awayTeam: 원정팀
  - 6. goals(homeTeam) : 홈팀이 넣은 골의 수
  - 7. goals(homeTeam) : 원정팀이 넣은 골의 수
  - 8. result : 경기 결과
  - 9. A(awayTeam 승리), D(Draw : 무승부), H(homeTeam 승리)로 구분
  - 10. halfTimeGoals(homeTeam) : 홈팀이 전반전에 넣은 골의 수
  - 11.halfTimeGoals(awayTeam) : 원정팀이 전반전에 넣은 골의 수
  - 12. shots(homeTeam) : 홈팀이 시도한 슈팅의 수
  - 13. shots(awayTeam) : 원정팀이 시도한 슈팅의 수
  - 14. shotsOnTarget(homeTeam) : 홈팀이 시도한 유효한 슈팅의 수
  - 15. shotsOnTarget(awayTeam) : 원정팀이 시도한 유효한 슈팅의 수
  - 16. corners(homeTeam) : 홈팀이 시도한 코너킥의 수
  - 17. corners(awayTeam) : 원정팀이 시도한 코너킥의 수
  - 18. fouls(homeTeam) : 홈팀이 받은 파울의 수
  - 19. fouls(awayTeam) : 원정팀이 받은 파울의 수
  - 20. yellowCards(homeTeam) : 홈팀이 받은 엘로카드의 수
  - 21. yellowCards(awayTeam) : 원정팀이 받은 엘로카드의 수
  - 22. redCards(homeTeam) : 홈팀이 받은 레드카드의 수
  - 23. redCards(homeTeam) : 원정팀이 받은 레드카드의 수

test.csv
- 미래에 발생할 EPL(England Premier League) 경기 목록
- 미래의 경기이기 때문에 경기 결과와 별도의 통계 정보가 존재하지 않음
  - 1. matchID : 경기 고유 ID
  - 2. season : 경기가 진행된 시즌
  - 3. date : 경기가 예정된 일자 (일부 데이터의 경우 데이터 수집환경에 따라 실제 영국 시간과 방생한 경기 차이가 존재함)
  - 4. homeTeam : 홈팀
  - 5. awayTeam : 원정팀

sample_submission.csv (제출 파일)
- 제출양식으로, 한 경기에서 발생할 수 있는 결과들에 해당되는 확률을 각각 예측
  - 1. matchID : 경기 고유 ID
  - 2. A : awayTeam이 승리할 확률을 예측
  - 3. D : 무승부가 발생할 확률을 예측
  - 4. H : homeTeam이 승리할 확률을 예측


### **코드 흐름**

**EDA**
- 코드에 따로 EDA는 나타나지 않음

**Feature Engineering & preprocessing**

Feature Engineering
- 데이터 로딩 함수 설정: load_train_valid()로 train.csv에서 마지막 88 경기를 validationset으로 분리하고, load_train_test()에 test.csv를 로드
- 팀 라벨 인코딩: test_encoding()함수를 보면, 홈팀 기준으로 승 수를 계산하여 순위를 부여하는 labelencoding을 진행한 것을 확인가능
- 지수이동평균 피처: EWMA()를 이용하여 주어진 columns에 대해 경기단위 지수이동평균을 계산
- 이상치 탐치: 15-85 percentile 기반 IQP로 이상치 인덱스를 반환함
- 팀 별 시간 기반 평균 통계 : 골/승률의 최근 N개의 경이 rolling 평균을 계산함
- 홈/어웨이 팀의 다른 스탯에 대한 평균 피처를 생성

Preprocessing
- 날짜 파싱. match feature 생성, label encoding, 피처를 통합한 하나의 함수 생성
- 불필요한 column 제거
- 주요 통계 feature 생성

**Seed**
- 실험의 재현성을 확보하기 위하여 randoon, numpy, Python의 해시값 등을 모두 고정하여 결과가 바뀌지 않도록 함

**Train/Test**
- 여기서 사용한 것은 Big6 구단을 설정해 6개의 구단에 속하면 1을 반환하고 아니면 0을 반환하는 함수를 만들고 이들을 분류하여 별도로 모델을 학습합
- 빅6 구단 여부에 따라 이들을 따로 모델을 학습하고, 승리하면 0, 무승부는 1, 지면 2를 반환하는 for문을 구성하고 이들의 결과를 저장함
**Submission**
- 제출파일 형성

###**새롭게 알게된 점**
- 지수이동 평균 피처

: 시계열 데이터나 누적정보가 존재하는 데이터에서 최근 값을 더욱 중요하게 반영하여 평균을 계산하는 기법으로, 흔히 주가 분석, 날씨, 경기력과 같은 변화가 시간에 따라 누적되는 데이터에서 자주 사용하는 연산

###**전반적인 코드 확인 후 느낀점**
1. 모델을 빅6구단에 속하는지, 소속 여부에 따라 모델을 분리하여 학습함
- 강팀과 약팀 간의 경기 양상의 구조적 차이를 더욱 잘 반영할 수 있을 것 같고, 이로 인해 두 개의 양상으로 데이터를 학습시킴으로써 모델이 더욱 확률을 잘 예측하도록 진행할 수 있음
  
  -> 이러한 방식은 데이터가 비선형적이거나 분리된 모델 간 특성이 뚜렷한 경우에 자주 이용하는 방법임  
2. 모델은 단순한 RogisticRegression을 진행하였으나, 전차리와 그룹핑을 상세하게 진행하여 변수를 세세히 인코딩하고 있기 때문에 결과에서 예측성능이 높게 나오는 것을 유추할 수 있음 -> 즉 데이터를 면밀하게 파악해야 한다는 점을 여기서 다시한번 느낄 수 있음